## 로컬 환경에서 PDF 파일 RAG 검색하기 3단계

__step4__
- PDF 문서 여러개 로드하여 임베딩 후 csv 파일로 저장
- csv 파일을 랭체인 FAISS 인덱싱하여 인덱스 파일 생성 및 저장
- 랭체인 프레임워크 적용하여 llm 검색 까지 구현
- 파일 3개 인덱싱 후 1개 추가하여 llm 검색 하기 구현

In [2]:
import os
import csv
import json
import ast
import faiss
import numpy as np
import pandas as pd
from langchain.document_loaders import DirectoryLoader, PyMuPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import FAISS
from langchain.docstore.document import Document
from langchain_core.embeddings import Embeddings
from sentence_transformers import SentenceTransformer

# 1. 문서 로드
loader = DirectoryLoader(
    'data',                         # data 폴더
    glob='*.pdf',                   # 모든 pdf 파일
    loader_cls=PyMuPDFLoader
)
docs = loader.load()

# 2. 문서 분할
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=50)
split_documents = text_splitter.split_documents(docs)

# 3. 임베딩을 위한 클래스 생성
class KoSentenceTransformerEmbeddings(Embeddings):
    def __init__(self, model_name):
        self.model = SentenceTransformer(model_name)        
    
    def embed_documents(self, texts):
        return self.model.encode(texts, convert_to_numpy=True).tolist()         # 문서 임베딩
    
    def embed_query(self, text):
        return self.model.encode([text], convert_to_numpy=True).tolist()[0]     # 쿼리 임베딩

embedding_model = KoSentenceTransformerEmbeddings("jhgan/ko-sroberta-multitask")    # 내가 임베딩 할 때 사용하는 모델 

# 4. 임베딩 파일 저장 
# 처리할 문서 리스트, 임베딩 모델, 저장할 폴더 경로 
def save_embeddings_to_csv(documents, embedding_model, folder_path):
    os.makedirs(folder_path, exist_ok=True)                             # 폴더가 없으면 생성, 있으면 넘어감
    
    file_docs = {}                                                      # pdf 파일별로 임베딩 결과를 저장할 딕셔너리 초기화 
    for doc in documents:
        file_name = os.path.basename(doc.metadata['source']).replace('.pdf', '')        # 원본 파일에서 제목만 추출 
        if file_name not in file_docs:
            file_docs[file_name] = []
        file_docs[file_name].append(doc)
    
    for file_name, docs in file_docs.items():
        full_path = os.path.join(folder_path, f"{file_name}.csv")
        
        # 문서 임베딩 데이터 생성
        embeddings = embedding_model.embed_documents([doc.page_content for doc in docs])
        
        # document 는 문서 chunk, embedding 은 임베딩 벡터 데이터 
        # 지정된 경로 full_path 에 파일을 쓰기 모드로 열기
        # newline : 윈도우와 동일한 줄바꿈 형식 설정 , utf-8 인코딩 설정
        with open(full_path, mode='w', newline='', encoding='utf-8') as file:
            writer = csv.writer(file)   # 파일을 작성할 수 있도록 객체 생성 
            writer.writerow(["document", "embedding"]) # csv 파일을 열고 document embedding 열 작성
            
            for doc, embedding in zip(docs, embeddings): # 문서와 임베딩 데이터를 한 쌍식 묶어서 반복 
                # doc.page_content : 문서의 실제 내용 
                # embedding : 임베딩 데이터를 json 문자열 형태로 저장
                writer.writerow([doc.page_content, json.dumps(embedding)])  
        
        print(f"임베딩 데이터가 {full_path} 파일에 저장되었습니다.")
    
# 임베딩 csv 파일 저장하는 함수 실행
save_embeddings_to_csv(split_documents, embedding_model, 'csv/')

# 5. CSV에서 임베딩 로드 및 FAISS 인덱스 생성
# csv 파일들이 들어있는 경로 
def load_csv_embeddings(folder_path):
    data_dict = {}  # csv 에서 불러온 데이터를 저장하는 딕셔너리 
    
    # folder_path 경로에 있는 모든 csv 파일 읽기 
    for filename in os.listdir(folder_path):
        if filename.endswith(".csv"): # 파일이 csv 확장인 경우에만 실행 
            file_path = os.path.join(folder_path, filename) # 파일의 전체 경로 
            df = pd.read_csv(file_path) # 해당  csv 파일을 padas 라이브러리 사용해서 읽기 
            
            # csv 파일이 document 와 embedding 이 존재하는지 확인하기 
            if "document" in df.columns and "embedding" in df.columns:
                documents, embeddings, metadatas = [], [], [] # 데이터 저장할 리스트 초기화
                for _, row in df.iterrows():
                    text = row["document"] # document에서 문서를 가져오기 
                    try:
                        embedding = json.loads(row["embedding"])  # JSON 로드 방식으로 변경
                        if isinstance(embedding, list):
                            embeddings.append(np.array(embedding, dtype=np.float32))
                            documents.append(text)
                            metadatas.append({"source": filename})
                        else:
                            print(f"⚠️ {filename}의 임베딩 형식이 올바르지 않습니다!")
                    except Exception as e:
                        print(f"{filename}에서 임베딩 변환 오류: {e}")
                data_dict[filename.replace('.csv', '')] = (documents, embeddings, metadatas)
    
    return data_dict

# FAISS 인덱스 생성 및 저장
def create_faiss_indexes(data_dict, save_path):
    os.makedirs(save_path, exist_ok=True)       # 폴더가 없는 경우 만들고, 있으면 지나감
    
    for file_name, (documents, embeddings, metadatas) in data_dict.items():
        # FAISS 인덱스를 생성
        # texts : 문서 텍스트 , embedding : 임베딩 모델, metadatas : 각 문서의 메타데이터
        vector_store = FAISS.from_texts(texts=documents, embedding=embedding_model, metadatas=metadatas)
        # 인덱스 파일 저장 경로 설정 
        faiss_index_path = os.path.join(save_path, file_name)
        vector_store.save_local(faiss_index_path) # 생성한 인덱스 파일 저장, .faiss 형식으로 저장됨
        print(f" {file_name}에 대한 FAISS 인덱스 저장 완료: {faiss_index_path}")

# 실행
if __name__ == "__main__":
    data_folder = "./csv"
    faiss_index_folder = "./faiss_index"
    
    data_dict = load_csv_embeddings(data_folder)
    print(f"{len(data_dict)}개의 파일 로드 완료!") # 로드된 csv 파일의 개수 출력 
    
    if data_dict:
        create_faiss_indexes(data_dict, faiss_index_folder)
    else:
        print("문서 또는 임베딩 데이터가 없습니다. FAISS 인덱스를 생성할 수 없습니다!")


/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


✅ 임베딩 데이터가 csv/SPRI_AI_Brief_2023년12월호_F.csv 파일에 저장되었습니다.
✅ 임베딩 데이터가 csv/AI기반_인파분석플랫폼구축_제안서.csv 파일에 저장되었습니다.
📄 2개의 파일 로드 완료!
 SPRI_AI_Brief_2023년12월호_F에 대한 FAISS 인덱스 저장 완료: ./faiss_index/SPRI_AI_Brief_2023년12월호_F
 AI기반_인파분석플랫폼구축_제안서에 대한 FAISS 인덱스 저장 완료: ./faiss_index/AI기반_인파분석플랫폼구축_제안서


In [13]:
# API 키를 환경변수로 관리하기 위한 설정 파일
import os
from dotenv import load_dotenv

# API 키 정보 로드
load_dotenv()

True

In [14]:
from langchain_core.prompts import PromptTemplate
from langchain_community.llms import Ollama
from langchain.chains import RetrievalQA
from langchain.vectorstores import FAISS

from langchain_openai import ChatOpenAI
import os

# ✅ Step 6: 프롬프트 생성
prompt = PromptTemplate.from_template(
    """You are an assistant for question-answering tasks. 
Use the following pieces of retrieved context to answer the question. 
If you don't know the answer, just say that you don't know. 
Answer in Korean, and make sure the answer ends with '입니다'.

#Context: 
{context}

#Question:
{question}

#Answer(Ensure the response ends with '입니다'):"""
)

# Step 7: 언어 모델 (LLM) 생성
# llm = Ollama(model="llama3.2")  
llm = ChatOpenAI(model_name="gpt-4o", temperature=0.5)


# ✅ Step 4: 모든 FAISS 인덱스를 로드하는 함수
def load_all_faiss_indexes(index_folder):
    faiss_indexes = {}

    for file_name in os.listdir(index_folder):
        index_path = os.path.join(index_folder, file_name)
        if os.path.isdir(index_path):  # 폴더 형태의 FAISS 인덱스인지 확인
            try:
                faiss_indexes[file_name] = FAISS.load_local(
                    index_path,
                    embedding_model,
                    allow_dangerous_deserialization=True  # 보안 옵션 추가
                )
                print(f"✅ {file_name} 인덱스 로드 완료!")
            except Exception as e:
                print(f"⚠️ {file_name} 인덱스 로드 실패: {e}")
    
    return faiss_indexes

# ✅ Step 5: 모든 FAISS 인덱스를 검색하는 함수
def search_across_all_indexes(question, faiss_indexes, top_k=5):
    all_docs = []

    # 모든 FAISS 인덱스에서 검색
    for index_name, index in faiss_indexes.items():
        retriever = index.as_retriever(search_type="similarity", search_kwargs={"k": top_k})
        docs = retriever.get_relevant_documents(question)  

        # 문서 출처 정보 추가
        for doc in docs:
            doc.metadata["source"] = index_name
        all_docs.extend(docs)

    if not all_docs:
        return "❌ 관련 문서를 찾을 수 없습니다."

    # ✅ Step 6: 검색된 문서들을 기반으로 컨텍스트 생성
    context = "\n\n".join([doc.page_content for doc in all_docs])
    
    # ✅ Step 7: LLM을 사용해 답변 생성
    formatted_prompt = prompt.format(context=context, question=question)
    answer = llm.invoke(formatted_prompt)

    # ✅ Step 8: 검색된 문서들의 출처 정리
    sources = "\n".join(set([doc.metadata.get("source", "알 수 없음") for doc in all_docs]))

    return f"✅ 답변: {answer.strip()}\n📁 출처:\n{sources}"

# ✅ 실행 코드
if __name__ == "__main__":
    faiss_index_folder = "./faiss_index"  # FAISS 인덱스 저장 폴더
    question = "삼성전자가 만든 ai의 이름이 뭐야?"

    # ✅ Step 1: 모든 FAISS 인덱스 로드
    faiss_indexes = load_all_faiss_indexes(faiss_index_folder)

    # ✅ Step 2: 질문 수행
    response = search_across_all_indexes(question, faiss_indexes)
    
    # ✅ Step 3: 결과 출력
    print(response)

✅ 2040_seoul 인덱스 로드 완료!
✅ 2024년 경기도 인구영향평가_편집본 인덱스 로드 완료!
✅ SPRI_AI_Brief_2023년12월호_F 인덱스 로드 완료!
✅ AI기반_인파분석플랫폼구축_제안서 인덱스 로드 완료!


AttributeError: 'AIMessage' object has no attribute 'strip'

In [6]:
from langchain_core.prompts import PromptTemplate
from langchain_community.llms import Ollama
from langchain.chains import RetrievalQA
from langchain.vectorstores import FAISS

import os

# ✅ Step 6: 프롬프트 생성
prompt = PromptTemplate.from_template(
    """You are an assistant for question-answering tasks. 
Use the following pieces of retrieved context to answer the question. 
If you don't know the answer, just say that you don't know. 
Answer in Korean, and make sure the answer ends with '입니다'.

#Context: 
{context}

#Question:
{question}

#Answer(Ensure the response ends with '입니다'):"""
)

# Step 7: 언어 모델 (LLM) 생성
llm = Ollama(model="llama3.2")  

# FAISS 인덱스 로드 (보안 옵션 추가)
faiss_index_folder = "./faiss_index"
faiss_indexes = {}

for file_name in os.listdir(faiss_index_folder):
    index_path = os.path.join(faiss_index_folder, file_name)
    if os.path.isdir(index_path):
        faiss_indexes[file_name] = FAISS.load_local(
            index_path,
            embedding_model,
            allow_dangerous_deserialization=True  # 보안 옵션 추가
        )

# 모든 인덱스를 검색하는 함수
def multi_index_search(question, faiss_indexes):
    all_docs = []
    
    # 각 FAISS 인덱스에서 검색
    for index_name, index in faiss_indexes.items():
        retriever = index.as_retriever(search_type="similarity", search_kwargs={"k": 3})  # 🔹 FAISS 검색 최적화
        docs = retriever.get_relevant_documents(question)  # 각 인덱스에서 검색된 문서
        for doc in docs:
            doc.metadata["source"] = index_name  # 출처 정보 추가
        all_docs.extend(docs)

    if not all_docs:
        return "관련 문서를 찾을 수 없습니다."
    
    # FAISS 검색 결과를 기반으로 RetrievalQA 생성
    retriever = FAISS.from_documents(all_docs, embedding_model).as_retriever()

    qa_chain = RetrievalQA.from_chain_type(
        llm=llm,
        retriever=retriever,  # 올바른 retriever 전달
        chain_type="stuff",
        return_source_documents=True,
        chain_type_kwargs={"prompt": prompt}
    )
    
    response = qa_chain.invoke({"query": question})  # context 제거 후 수정
    
    answer = response.get("result", "⚠️ 답변을 생성할 수 없습니다.")  # key 수정
    sources = "\n".join(set([doc.metadata.get("source", "알 수 없음") for doc in all_docs]))

    return f" 답변: {answer}\n 출처: {sources}"

def search_across_all_indexes(question, faiss_indexes):
    all_docs = []

    # 모든 FAISS 인덱스에서 검색
    for index_name, index in faiss_indexes.items():
        retriever = index.as_retriever(search_type="similarity", search_kwargs={"k": 3})  # 🔹 FAISS 검색 최적화
        docs = retriever.get_relevant_documents(question)  # 각 인덱스에서 검색된 문서 가져오기

        # 문서 출처 정보 추가
        for doc in docs:
            doc.metadata["source"] = index_name  # 문서 출처 추가
        all_docs.extend(docs)

    if not all_docs:
        return "❌ 관련 문서를 찾을 수 없습니다."

    # 검색된 문서들의 내용을 합쳐서 LLM에 전달
    context = "\n\n".join([doc.page_content for doc in all_docs])
    formatted_prompt = prompt.format(context=context, question=question)

    # LLM을 사용해 답변 생성
    answer = llm.invoke(formatted_prompt)

    # 문서 출처 정리
    sources = "\n".join(set([doc.metadata.get("source", "알 수 없음") for doc in all_docs]))

    return f"✅ 답변: {answer}\n📁 출처:\n{sources}"



# 사용 예시
question = "인구정책사업"
# print(multi_index_search(question, faiss_indexes))
print(search_across_all_indexes(question, faiss_indexes))


✅ 답변: 인구정책사업입니다.
📁 출처:
SPRI_AI_Brief_2023년12월호_F
2040_seoul
AI기반_인파분석플랫폼구축_제안서
2024년 경기도 인구영향평가_편집본


In [7]:
def search_faiss_index(query, index_folder, file_name, top_k=5):
    index_path = os.path.join(index_folder, file_name)
    
    # FAISS 인덱스 로드 시 allow_dangerous_deserialization 옵션 추가
    vector_store = FAISS.load_local(index_path, embedding_model, allow_dangerous_deserialization=True)
    
    query_embedding = embedding_model.embed_query(query)
    results = vector_store.similarity_search_by_vector(query_embedding, k=top_k)
    
    print(f"🔍 '{query}' 검색 결과:")
    for i, result in enumerate(results):
        print(f"{i+1}. {result.page_content} (출처: {result.metadata['source']})")
    
    return results

# search_faiss_index("포항에서 열리는 축제의 이름은 무엇인가?", "./faiss_index", "AI기반_인파분석플랫폼구축_제안서")
# search_faiss_index("삼성에서 만든 AI의 이름은 무엇인가?", "./faiss_index", "SPRI_AI_Brief_2023년12월호_F")
search_faiss_index("경기도 인구정책사업에서 현행 유지 평가를 받은 사업의 개수는?", "./faiss_index", "2024년 경기도 인구영향평가_편집본")

🔍 '경기도 인구정책사업에서 현행 유지 평가를 받은 사업의 개수는?' 검색 결과:
1. - 18 -
4. 경기도 인구영향평가 정성평가 결과
 □ 정성평가 결과
○ 48개 인구정책사업에 대해 인구영향평가를 실시하였으며 정성평가는 ‘현행 유지’, ‘일부 개선’, 
‘전면 개선’의 세 가지 방식으로 시행되었음
○ 사업 중 ‘현행유지’ 평가를 받은 인구정책사업은 4개로 ‘취약지 당직의료기관 지원’, ‘경기도 어
린이 건강과일 공급’, ‘가족친화 사회환경 조성’, ‘일생활균형 상담지원’이었음
○ ‘현행유지’ 평가를 받은 4개 사업을 제외한 나머지 44개 사업은 ‘일부 개선’ 평가를 받았으며, 
44개 사업은 개선에 대한 의견수렴을 실시함. ‘전면 개선’ 평가를 받은 인구정책사업은 없었음
○ 인구영향평가 결과에 대한 사업부서의 피드백 결과를 살펴보면, 44개 사업 중(4개 사업은 ‘현
행유지’ 평가) 31개 사업이 인구영향평가 결과를 반영할 예정이고, 나머지 10개 사업에 대해
서는 인구영향평가 결과 미반영 사유가 제출되었음
 □ 사업별 전문가 총평 요약
구분
사업명
전문가 총평
1
어르신 안전 
하우징
ž
노인 인구 증가에 따른 노인 맞춤형 주거 환경 개선을 위한 정책은 앞으로 더욱더 
확대되어야 하며, 자가 소유자 외에도 임차인도 정책의 수혜자가 될 수 있도록 할 필
요성이 있음
2
빈집정비 
지원사업
ž
인구 감소와 고령화로 인해 향후 빈집은 급속히 증가할 것이지만 재산권 등에 대한 
문제로 이에 대한 접근이 쉽지 않은 상황으로 인구구조 변화에 대응하는 중요한 정책
으로 볼 수 있음
ž
재정 투입만으로 빈집을 모두 정비하는 것은 불가능하므로 재정 운용 방향에 대한 고
민이 필요하다고 보여지며, 도민의 공감대 형성과 적극적인 홍보가 필요함
3
경기도 
노인종합상담
센터 지원
ž
노인 인구 증가로 2025년에 초고령사회에 진입할 것으로 예상되고, 노인 빈곤율과 자
살률은 OECD 국가 중 1위인 상황에서 노인들에게 심리상담 서비스를 제공하고, 노인 
상담의 전문성을 제

[Document(id='123853cc-ac8c-4e6a-84bd-eb51fdce59ea', metadata={'source': '2024년 경기도 인구영향평가_편집본.csv'}, page_content='- 18 -\n4. 경기도 인구영향평가 정성평가 결과\n □ 정성평가 결과\n○ 48개 인구정책사업에 대해 인구영향평가를 실시하였으며 정성평가는 ‘현행 유지’, ‘일부 개선’, \n‘전면 개선’의 세 가지 방식으로 시행되었음\n○ 사업 중 ‘현행유지’ 평가를 받은 인구정책사업은 4개로 ‘취약지 당직의료기관 지원’, ‘경기도 어\n린이 건강과일 공급’, ‘가족친화 사회환경 조성’, ‘일생활균형 상담지원’이었음\n○ ‘현행유지’ 평가를 받은 4개 사업을 제외한 나머지 44개 사업은 ‘일부 개선’ 평가를 받았으며, \n44개 사업은 개선에 대한 의견수렴을 실시함. ‘전면 개선’ 평가를 받은 인구정책사업은 없었음\n○ 인구영향평가 결과에 대한 사업부서의 피드백 결과를 살펴보면, 44개 사업 중(4개 사업은 ‘현\n행유지’ 평가) 31개 사업이 인구영향평가 결과를 반영할 예정이고, 나머지 10개 사업에 대해\n서는 인구영향평가 결과 미반영 사유가 제출되었음\n □ 사업별 전문가 총평 요약\n구분\n사업명\n전문가 총평\n1\n어르신 안전 \n하우징\nž\n노인 인구 증가에 따른 노인 맞춤형 주거 환경 개선을 위한 정책은 앞으로 더욱더 \n확대되어야 하며, 자가 소유자 외에도 임차인도 정책의 수혜자가 될 수 있도록 할 필\n요성이 있음\n2\n빈집정비 \n지원사업\nž\n인구 감소와 고령화로 인해 향후 빈집은 급속히 증가할 것이지만 재산권 등에 대한 \n문제로 이에 대한 접근이 쉽지 않은 상황으로 인구구조 변화에 대응하는 중요한 정책\n으로 볼 수 있음\nž\n재정 투입만으로 빈집을 모두 정비하는 것은 불가능하므로 재정 운용 방향에 대한 고\n민이 필요하다고 보여지며, 도민의 공감대 형성과 적극적인 홍보가 필요함\n3\n경기도 

In [8]:
from langchain.chat_models import ChatOpenAI
from langchain.chains import RetrievalQA
from langchain.prompts import PromptTemplate
from langchain.vectorstores import FAISS
import os

# 🔍 FAISS 인덱스를 검색하여 결과 반환
def search_faiss_index(query, index_folder, file_name, top_k=5):
    index_path = os.path.join(index_folder, file_name)
    
    # FAISS 인덱스 로드 (안전한 역직렬화 옵션 적용)
    vector_store = FAISS.load_local(index_path, embedding_model, allow_dangerous_deserialization=True)
    
    query_embedding = embedding_model.embed_query(query)
    results = vector_store.similarity_search_by_vector(query_embedding, k=top_k)
    
    print(f"🔍 '{query}' 검색 결과:")
    for i, result in enumerate(results):
        print(f"{i+1}. {result.page_content} (출처: {result.metadata['source']})")
    
    return results

# 🚀 LLM을 사용하여 질문 수행
def query_llm_with_context(query, index_folder, file_name, model_name="gpt-4"):
    # 1️⃣ FAISS 인덱스에서 관련 문서 검색
    results = search_faiss_index(query, index_folder, file_name, top_k=5)
    
    # 2️⃣ 검색된 문서를 컨텍스트로 결합
    context = "\n\n".join([result.page_content for result in results])
    
    # 3️⃣ LangChain 프롬프트 템플릿 정의
    prompt_template = PromptTemplate(
        input_variables=["context", "query"],
        template=(
            "다음은 관련 문서들입니다:\n\n{context}\n\n"
            "위 정보를 기반으로 다음 질문에 답변해 주세요:\n\n{query}"
        )
    )
    
    # 4️⃣ LLM 모델 초기화 (OpenAI API 사용)
    llm = Ollama(model="llama3.2")  
    
    # 5️⃣ QA 체인 생성
    qa_chain = RetrievalQA.from_chain_type(llm, retriever=None)
    
    # 6️⃣ LLM 호출하여 답변 생성
    final_prompt = prompt_template.format(context=context, query=query)
    response = qa_chain.run(final_prompt)
    
    print("\n🤖 AI의 답변:")
    print(response)
    
    return response

# 🔥 실행 예제
if __name__ == "__main__":
    index_folder = "./faiss_index"
    file_name = "sample_data"  # FAISS 인덱스 파일명
    user_query = "이 문서에서 중요한 내용은 무엇인가요?"

    query_llm_with_context(user_query, index_folder, file_name)


RuntimeError: Error in faiss::FileIOReader::FileIOReader(const char *) at /Users/runner/work/faiss-wheels/faiss-wheels/faiss/faiss/impl/io.cpp:68: Error: 'f' failed: could not open faiss_index/sample_data/index.faiss for reading: No such file or directory

In [ ]:
# 새롭게 추가할 폴더명
new_folder = "data/gg"

# 1. 새 폴더의 문서 로드
new_loader = DirectoryLoader(
    new_folder,                     # aaa 폴더
    glob="*.pdf",                   # 모든 pdf 파일
    loader_cls=PyMuPDFLoader
)
new_docs = new_loader.load()

# 2. 문서 분할 (기존 방식 사용)
new_split_documents = text_splitter.split_documents(new_docs)

# 3. 새로운 문서 임베딩을 csv/aaa 폴더에 저장
save_embeddings_to_csv(new_split_documents, embedding_model, "csv/gg")


In [ ]:
# 새로 추가된 aaa 폴더의 CSV 파일을 불러오기
new_data_dict = load_csv_embeddings("./csv/gg")

# 기존 FAISS 인덱스 폴더
faiss_index_folder = "./faiss_index"

# 새로 추가된 인덱스들을 기존 인덱스에 병합하는 함수
def append_faiss_indexes(existing_index_folder, new_data_dict):
    # 기존 인덱스 불러오기
    existing_indexes = {}

    for file_name in os.listdir(existing_index_folder):
        index_path = os.path.join(existing_index_folder, file_name)
        if os.path.isdir(index_path):
            existing_indexes[file_name] = FAISS.load_local(
                index_path,
                embedding_model,
                allow_dangerous_deserialization=True
            )

    # 새로운 데이터 처리
    for file_name, (documents, embeddings, metadatas) in new_data_dict.items():
        new_faiss_index = FAISS.from_texts(
            texts=documents,
            embedding=embedding_model,
            metadatas=metadatas
        )
        
        # 기존 인덱스가 있으면 병합
        if file_name in existing_indexes:
            existing_indexes[file_name].merge_from(new_faiss_index)
            print(f"기존 인덱스 {file_name}에 새로운 데이터 병합 완료!")
        else:
            existing_indexes[file_name] = new_faiss_index
            print(f"새로운 인덱스 {file_name} 생성 완료!")

    # 저장 (덮어쓰기)
    for file_name, index in existing_indexes.items():
        index_path = os.path.join(existing_index_folder, file_name)
        index.save_local(index_path)
        print(f"{file_name} FAISS 인덱스 업데이트 완료!")

# 실행
append_faiss_indexes(faiss_index_folder, new_data_dict)
